In [9]:
import os
import openai
import json

In [2]:
#Load API keys saved in keys.txt
keyfile = open("keys.txt", "r")
line = keyfile.readline()
line = line.strip()
openai.api_key = line


In [5]:
#DOCS: https://beta.openai.com/docs/api-reference?lang=python

#Engines:
#davinci
#curie
#babbage
#ada
response = openai.Completion.create(
    engine="davinci",
    temperature = 0.0,
    top_p = 1,
    prompt="This is a test",
    max_tokens=50,
    stop='\n'
    )

In [6]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " of the emergency broadcast system. This is only a test.\u201d"
    }
  ],
  "created": 1619094699,
  "id": "cmpl-2rSmZR57R5bTT00ftXRqMucHe1uaX",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}


In [38]:
response_dict = response.to_dict()
text = response_dict["choices"][0]["text"]

In [39]:
print(text)

 of the emergency broadcast system. This is only a test.”


In [ ]:
def loadDataset(datasetPath):
    dataset = open(datasetPath, 'w')